# Image testing for pre-processing

## Importing Modules

In [1]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

import csv
import json
from os import listdir
from os.path import isfile, join

from ManageLeafData import ManageLeafData as mld

## Loading image

In [2]:
# Global path must not be changed unless machine has been changed.
global_path = 'Collected Leaf data/Our Collection/Natural/'
# Local path can be changed anywhere, so before using it, specify the required path.
local_path = ''

In [3]:
img_1 = cv.imread(global_path + 'IMG_20230922_171310509.jpg')
cv.imshow('Raw Image', img_1)
cv.waitKey(0)
cv.destroyAllWindows()

In [4]:
image_lst = ['IMG_20230922_171255218.jpg', 'IMG20230922165200.jpg', 'IMG20230922170108.jpg', 'IMG20230922170842.jpg', 'IMG20231004155721.jpg']
# In this list the last image is from Botany collection, thus its path must be change.

## Image Smoothening

In [4]:
def GaussianSmoothing(mat):
    _pi = 3.14
    _sd = np.std(mat)
    _e = 2.718
    for y in range(len(mat)):
        for x in range(len(mat[y])):
            try:
                mat[y][x] = ((1 / (2 * _pi * (_sd ** 2)))) * (_e ** (((x ** 2) + (y ** 2)) * (1 / (2 * (x ** 2)))))
            except ZeroDivisionError:
                pass
    return mat

In [5]:
# img_1_myGblur = GaussianSmoothing(cv.cvtColor(img_1, cv.COLOR_BGR2GRAY))

# img_1_myGblur_resized = cv.resize(img_1_myGblur, (1000, 1000))
# cv.imshow('Image Gaussian Blur & Resized', img_1_myGblur_resized)
# cv.waitKey(0)
# cv.destroyAllWindows()

In [7]:
# img_1_resized_Gblur = cv.GaussianBlur(img_1_resized, (7,7), 0)
# cv.imshow('Image Resized & Gaussian Blur', img_1_resized_Gblur)
# cv.waitKey(0)
# cv.destroyAllWindows()

img_1_Gblur = cv.GaussianBlur(img_1, (7,7), 0)
cv.imshow('Image Gaussian Blur', img_1_Gblur)
cv.waitKey(0)
cv.destroyAllWindows()

## Image Resizing

In [8]:
# img_1_resized = cv.resize(img_1, (int(img_1.shape[1] / 6), int(img_1.shape[0] / 6)))
# cv.imshow('Image Resized', img_1_resized)
# cv.waitKey(0)
# cv.destroyAllWindows()

img_1_Gblur_resized = cv.resize(img_1_Gblur, (1000, 1000))
cv.imshow('Image Gaussian Blur & Resized', img_1_Gblur_resized)
cv.waitKey(0)
cv.destroyAllWindows()

## Gray Scaling

In [9]:
img_1_Gblur_resized_gray = cv.cvtColor(img_1_Gblur_resized, cv.COLOR_BGR2GRAY)
cv.imshow('Image Gaussian Blur & Resized & Gray', img_1_Gblur_resized_gray)
cv.waitKey(0)
cv.destroyAllWindows()

In [10]:
# Function to map each intensity level to output intensity level. 
def pixelVal(pix, r1, s1, r2, s2): 
    if (0 <= pix and pix <= r1): 
        return (s1 / r1)*pix 
    elif (r1 < pix and pix <= r2): 
        return ((s2 - s1)/(r2 - r1)) * (pix - r1) + s1 
    else: 
        return ((255 - s2)/(255 - r2)) * (pix - r2) + s2 

# Define parameters. 
r1 = 110 # controls darkness
s1 = 0.15 # lower bound
r2 = 7 # controls lightness
s2 = 250 # upper bound

# Vectorize the function to apply it to each value in the Numpy array. 
pixelVal_vec = np.vectorize(pixelVal) 

# Apply contrast stretching. 
img_1_contrast_stretched = pixelVal_vec(img_1_Gblur_resized_gray, r1, s1, r2, s2) 

cv.imshow('Contrast Stretching', img_1_contrast_stretched)
cv.waitKey(0)
cv.destroyAllWindows()


In [11]:
img_thresh = cv.threshold(img_1_Gblur_resized_gray, 127, 255, cv.THRESH_BINARY)
cv.imshow('Thresholding', img_thresh[1])
cv.waitKey(0)
cv.destroyAllWindows()

# print(img_thresh)

In [68]:
img_adptthresh = cv.adaptiveThreshold(img_1_Gblur_resized_gray, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY, 199, 0)
cv.imshow('Adaptive Thresholding', img_adptthresh)
cv.waitKey(0)
cv.destroyAllWindows()

In [98]:
img_adptthresh1 = cv.adaptiveThreshold(img_1_Gblur_resized_gray, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY, 199, 3.5)
cv.imshow('Adaptive Thresholding 1', img_adptthresh1)
# img_adptthresh2 = cv.adaptiveThreshold(img_1_Gblur_resized_gray, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY, 199, 4)
# cv.imshow('Adaptive Thresholding 2', img_adptthresh2)
img_adptthresh3 = cv.adaptiveThreshold(img_1_Gblur_resized_gray, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY, 1001, 3.5)
cv.imshow('Adaptive Thresholding 3', img_adptthresh3)
cv.waitKey(0)
cv.destroyAllWindows()

In [25]:
img_hist = cv.equalizeHist(img_thresh[1])
cv.imshow('Histogram equlization', img_hist)
cv.waitKey(0)
cv.destroyAllWindows()

## Combining these processes in a single function

In [5]:
def ImagePreprocess_Combination_1(name: str):
    cv.imshow('Final Image', pixelVal_vec(cv.resize(cv.GaussianBlur(cv.imread(name)
                                           , (7,7), 0), 
                           (1000, 1000)), 
                r1, s1, r2, s2))
    cv.waitKey(0)
    cv.destroyAllWindows()

In [6]:
def ImagePreprocess_Combination_2(name: str):
    cv.imshow('Final Image', cv.equalizeHist(cv.adaptiveThreshold(cv.cvtColor(cv.resize(cv.GaussianBlur(cv.imread(name)
                                                                                                        , (7,7), 0)
                                                                                        , (1000, 1000))
                                                                              , cv.COLOR_BGR2GRAY)
                                                                  , 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY, 199, 5)
                                            )
             )
    cv.waitKey(0)
    cv.destroyAllWindows()

In [31]:
for image in image_lst:
    ImagePreprocess_Combination_2(global_path + image)

error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\smooth.dispatch.cpp:617: error: (-215:Assertion failed) !_src.empty() in function 'cv::GaussianBlur'


In [7]:
import pandas as pd

df = pd.read_csv('Leaf_Data_Mapped.csv')
df

,Image,Common name,Scientific name,Class number
0,IMG20231004154634.jpg,Peepla,Ficus religiosa,0
1,IMG20231004154651.jpg,Peepla,Ficus religiosa,0
2,IMG20231004154748.jpg,Peepla,Ficus religiosa,0
3,IMG20231004154809.jpg,Peepla,Ficus religiosa,0
4,IMG20231004154917.jpg,Peepla,Ficus religiosa,0
...,...,...,...,...
92,IMG20231004172618.jpg,Pipal,Ficus amplissima,19
93,IMG20231004172657.jpg,Pipal,Ficus amplissima,19
94,IMG20231004172723.jpg,Pipal,Ficus amplissima,19
95,IMG20231004172803.jpg,Pipal,Ficus amplissima,19


In [35]:
local_path = 'Collected Leaf data/Botany Collection/Natural/'
_cnt = 0
for image in df['Image']:
    ImagePreprocess_Combination_2(local_path + image)
    if _cnt == 10: break
    _cnt += 1

In [8]:
image_blackBG_lst = mld.getImageNameList('BlackBG/')
image_blackBG_lst

['IMG_20231004_161936.jpg',
 'IMG_20231004_162013.jpg',
 'IMG_20231004_162043.jpg',
 'IMG_20231004_164637.jpg',
 'IMG_20231004_171344.jpg',
 'IMG_20231004_171917.jpg',
 'IMG_20231004_173050.jpg',
 'IMG_20231004_203638.jpg',
 'IMG_20231004_211127.jpg',
 'IMG_20231004_211747.jpg']

In [37]:
local_path = 'BlackBG/'
for image in image_blackBG_lst:
    ImagePreprocess_Combination_2(local_path + image)

# NEXT TASK

1) Train a model on the black background images and also pre-process them along with segmentation.
2) Create a prog that takes natural images, then creates multiple combinations with varying parameters of thresholding func, then puts the processed image into the model and gets the accuracy for any single class.
3) Now train that model with that new image.

=============================================================================================================================

# EXPERIMENTS

=============================================================================================================================

## Mapping completed and stored in Leaf_Data_Mapped.csv file

-----------------------------------------------------------------------------------------------------------------------------

-----------------------------------------------------------------------------------------------------------------------------